# ランキングと転倒数

Code: [ranking_and_inversion.py](https://github.com/Kyoroid/algorithm/blob/master/lib/misc/ranking_and_inversion.py)  
Test: [test_ranking_and_inversion.py](https://github.com/Kyoroid/algorithm/blob/master/test/misc/test_ranking_and_inversion.py)

## ランキング・順序付け

https://en.wikipedia.org/wiki/Ranking


ランキング (ranking) または順序付けは、順列のある値が「何番目に小さいか(順位)」を求めることをいう。  
同じ値に同じ順位を割り当てる順位付けを dense ranking、同じ値同士に対しても一貫した規則で順序付けしたものを ordinal ranking という。  

特に Ordinal ranking は順序関係を保ったまま、順列を小さな値に置き換えることができるため、座標圧縮などに利用される。

## 転倒数

順列中のある2数に着目したとき、出現位置の大小関係と値の大小関係が一致していない組を「転倒」という。  
例えば $\{1, 4, 9, 6, 11\}$ という数列において、2番目と3番目 (0-indexed) の値は自然な順番と逆順に並んでいる。このとき、$(i, j) = (2, 3)$ は転倒の1つである。  
順列中の転倒の総和を「転倒数」という。

また、それぞれの要素について、右側にある要素の転倒数を数えて作った数列を「転倒ベクトル」という。

$$
V_i = \{(A_i, A_k) | (i \lt k) \wedge (A_i \gt A_k)\}
$$

転倒ベクトルの要素の和が転倒数となる。

## 実装

$0, 1, \ldots n-1$ の序数を並び替えてできた数列 $A$ の転倒ベクトル $V_i$ は次の手順で求められる。  

1. 過去にある値が出現したかどうかを管理するための、長さ $n$ のブール配列 $flag$ を用意する。
1. 数列の右から $i=n-1, n-2, \ldots$ の順に走査し、以下を繰り返す。
    1. $i$ 番目の値 $A_i$ を見る。
    1. $A_i$ が出現したことを表すためのフラグを立てる。つまり $flag_{A_i} = 1$ とする。
    1. $A_i$ より小さな値が $A_i$ の右側にいくつ存在するか調べ、数え上げる。

「$A_i$ より小さな値が $A_i$ の右側にいくつ存在するか」について、右側の数字はすべて $flag$ に記録されている。  
よって、$[0, A_i)$ の範囲でいくつフラグが立っているかを数えれば、$A_i$ の右側に存在する $A_i$ より小さな値の個数を得ることができる。

フラグの計数は愚直に数えると $\mathcal{O}(n)$ だが、Fenwick Treeなどを使えば $\mathcal{O}(\log{n})$ に落ちる。  
そのため転倒ベクトルは $\mathcal{O}(n\log{n})$ で求まる。

任意の順列に対しても、順序付けをして $0, 1, \ldots n-1$ の序数を並び替えた数列に変換すれば、以降は同じ手順で転倒ベクトルを求められる。

## コード (Numpy + Numba)

In [1]:
import numpy as np
from numba import njit, i8
from lib.structure.fenwick_tree import FenwickTree


@njit("i8[:](i8[:])", cache=True)
def rank_vector(a: i8[:]) -> i8[:]:
    """配列の各要素の順位を求めます。同じ値をもつ要素同士は、もとの順序を保ったまま順位づけされます。

    Parameters
    ----------
    a : i8[:]
        配列

    Returns
    -------
    i8[:]
        配列の各要素の順位
    """
    order = np.argsort(a, kind="mergesort")
    rank = np.argsort(order, kind="mergesort")
    return rank



@njit("i8[:](i8[:])", cache=True)
def inversion_vector(a: i8[:]) -> i8[:]:
    """配列の転倒ベクトル V[i] = #{(A[i], A[k]) | i < k and A[i] >= A[k]} を求めます。

    Parameters
    ----------
    a : i8[:]
        配列

    Returns
    -------
    i8[:]
        転倒ベクトル
    """
    rank = rank_vector(a)
    ft = FenwickTree(a.size)
    inv_vec = np.zeros_like(a)
    for i in range(ft.n - 1, -1, -1):
        ri = rank[i]
        inv_vec[i] = ft.sum(0, ri)
        ft.add(ri, 1)
    return inv_vec


## 実行例

In [2]:
A = np.array([3, 1, 5, 4, 2], dtype=np.int64)
inv_vec = inversion_vector(A)
# (i, j) = {(0, 1), (0, 4), (2, 3), (2, 4), (3, 4)} are inversions.
print(*inv_vec)
print(inv_vec.sum())

2 0 2 1 0
5
